In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
# import datasets

transp = pd.read_csv("new_transportation_data.csv")
ejscreen = pd.read_csv("ejscreen_ga.csv")
eji = pd.read_csv("EJI_2024_United_States_CSV/EJI_2024_United_States.csv")
svi = pd.read_csv("svi_ga.csv")

In [14]:
#find column names for fips code (to merge on)

print(transp.columns)
print(ejscreen.columns)
print(eji.columns)
print(svi.columns)


Index(['Unnamed: 0', 'census_id', 'county', 'population', 'pm25',
       'pctile_pm25', 'diesel_pm', 'pctile_diesel', 'traffic_proximity',
       'pctile_traffic', 'commute_time', 'pctile_commute',
       'pct_income_transport', 'pctile_pct_inc_transport', 'geq90_pm25',
       'geq90_diesel', 'geq90_traffic', 'geq90_commute',
       'geq90_inc_transport'],
      dtype='object')
Index(['OID_', 'ID', 'STATE_NAME', 'ST_ABBREV', 'CNTY_NAME', 'REGION',
       'ACSTOTPOP', 'ACSIPOVBAS', 'ACSEDUCBAS', 'ACSTOTHH',
       ...
       'T_D2_DWATER', 'T_D5_DWATER', 'AREALAND', 'AREAWATER', 'NPL_CNT',
       'TSDF_CNT', 'EXCEED_COUNT_80', 'EXCEED_COUNT_80_SUP', 'Shape_Length',
       'Shape_Area'],
      dtype='object', length=230)
Index(['STATEFP', 'COUNTYFP', 'TRACTCE', 'AFFGEOID', 'GEOID', 'GEOID_2020',
       'COUNTY', 'StateDesc', 'STATEABBR', 'LOCATION',
       ...
       'E_AIAN', 'NHPI', 'E_NHPI', 'TWOMORE', 'E_TWOMORE', 'OTHERRACE',
       'E_OTHERRACE', 'Tribe_PCT_Tract', 'Tribe_Names', '

In [15]:
#create dataframe of SVI values by Georgia county
svi_values_all_states = svi[["STATE", "COUNTY", "FIPS", "RPL_THEMES"]]
svi_values_ga = svi_values_all_states.query("STATE == 'Georgia'")
svi_values_ga = svi_values_ga.rename(columns={"RPL_THEMES" : "SVI"})
svi_values_ga

#subset eji dataset for Georgia only
eji_ga = eji.query("StateDesc == 'Georgia'")
eji_ga

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,GEOID_2020,COUNTY,StateDesc,STATEABBR,LOCATION,...,E_AIAN,NHPI,E_NHPI,TWOMORE,E_TWOMORE,OTHERRACE,E_OTHERRACE,Tribe_PCT_Tract,Tribe_Names,Tribe_Flag
11581,13,31,110201,140000US13031110201,13031110201,13031110201,Bulloch County,Georgia,GA,Census Tract 1102.01; Bulloch County; Georgia,...,0.0,0,0.0,140,3.8,28,0.8,0.0,-999,-999
11582,13,31,110302,140000US13031110302,13031110302,13031110302,Bulloch County,Georgia,GA,Census Tract 1103.02; Bulloch County; Georgia,...,0.0,0,0.0,96,2.0,0,0.0,0.0,-999,-999
11583,13,33,950101,140000US13033950101,13033950101,13033950101,Burke County,Georgia,GA,Census Tract 9501.01; Burke County; Georgia,...,0.0,0,0.0,0,0.0,0,0.0,0.0,-999,-999
11584,13,33,950200,140000US13033950200,13033950200,13033950200,Burke County,Georgia,GA,Census Tract 9502; Burke County; Georgia,...,0.0,0,0.0,77,2.3,0,0.0,0.0,-999,-999
11585,13,45,910502,140000US13045910502,13045910502,13045910502,Carroll County,Georgia,GA,Census Tract 9105.02; Carroll County; Georgia,...,0.8,0,0.0,0,0.0,133,4.1,0.0,-999,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39541,13,151,70119,140000US13151070119,13151070119,13151070119,Henry County,Georgia,GA,Census Tract 701.19; Henry County; Georgia,...,0.0,0,0.0,169,2.5,0,0.0,0.0,-999,-999
39542,13,151,70120,140000US13151070120,13151070120,13151070120,Henry County,Georgia,GA,Census Tract 701.20; Henry County; Georgia,...,0.0,0,0.0,0,0.0,0,0.0,0.0,-999,-999
39543,13,151,70123,140000US13151070123,13151070123,13151070123,Henry County,Georgia,GA,Census Tract 701.23; Henry County; Georgia,...,0.0,0,0.0,27,1.0,0,0.0,0.0,-999,-999
39544,13,151,70207,140000US13151070207,13151070207,13151070207,Henry County,Georgia,GA,Census Tract 702.07; Henry County; Georgia,...,0.0,0,0.0,109,3.9,0,0.0,0.0,-999,-999


In [16]:
# Standardize county names by adding " County" to transp and ejscreen
transp["county"] = transp["county"] + " County"


In [17]:
# Merge datasets on fips code
# (transp - census_id) (svi - FIPS) (eji - GEOID_2020) (ejscreen_ga - ID)
merged_fips = transp.merge(svi_values_ga, left_on="census_id", right_on="FIPS")
merged_fips = merged_fips.merge(eji_ga, left_on="census_id", right_on="GEOID_2020")
merged_fips = merged_fips.merge(ejscreen, left_on="census_id", right_on="ID")
merged_fips.to_csv("merged_fips.csv", index=False)

In [ ]:
#create list of variables (extracting column names)

def extract_columns(csv_file, output_excel="column_names.xlsx"):
    # Read CSV file
    df = pd.read_csv(csv_file)
    
    # Extract column names
    column_names = df.columns.tolist()
    
    # Print column names as a list
    print("Column Names:")
    print(column_names)
    
    # Save column names to an Excel file
    column_df = pd.DataFrame({'Column Names': column_names})
    column_df.to_excel(output_excel, index=False)
    print(f"Column names exported to {output_excel}")
    
    return column_names

extract_columns("merged_fips.csv")


Column Names:
['Unnamed: 0', 'census_id', 'county', 'population', 'pm25', 'pctile_pm25', 'diesel_pm', 'pctile_diesel', 'traffic_proximity', 'pctile_traffic', 'commute_time', 'pctile_commute', 'pct_income_transport', 'pctile_pct_inc_transport', 'geq90_pm25', 'geq90_diesel', 'geq90_traffic', 'geq90_commute', 'geq90_inc_transport', 'STATE', 'COUNTY_x', 'FIPS', 'SVI', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'AFFGEOID', 'GEOID', 'GEOID_2020', 'COUNTY_y', 'StateDesc', 'STATEABBR', 'LOCATION', 'E_TOTPOP', 'M_TOTPOP', 'E_DAYPOP', 'SPL_EJI', 'RPL_EJI', 'SPL_SER', 'RPL_SER', 'SPL_EJI_CBM', 'RPL_EJI_CBM', 'E_MINRTY', 'EPL_MINRTY', 'SPL_SVM_DOM1', 'RPL_SVM_DOM1', 'E_POV200', 'EPL_POV200', 'E_NOHSDP', 'EPL_NOHSDP', 'E_UNEMP', 'EPL_UNEMP', 'E_RENTER', 'EPL_RENTER', 'E_HOUBDN', 'EPL_HOUBDN', 'E_UNINSUR', 'EPL_UNINSUR', 'E_NOINT', 'EPL_NOINT', 'SPL_SVM_DOM2', 'RPL_SVM_DOM2', 'E_AGE65', 'EPL_AGE65', 'E_AGE17', 'EPL_AGE17', 'E_DISABL', 'EPL_DISABL', 'E_LIMENG', 'EPL_LIMENG', 'SPL_SVM_DOM3', 'RPL_SVM_DOM3', 'E

['Unnamed: 0',
 'census_id',
 'county',
 'population',
 'pm25',
 'pctile_pm25',
 'diesel_pm',
 'pctile_diesel',
 'traffic_proximity',
 'pctile_traffic',
 'commute_time',
 'pctile_commute',
 'pct_income_transport',
 'pctile_pct_inc_transport',
 'geq90_pm25',
 'geq90_diesel',
 'geq90_traffic',
 'geq90_commute',
 'geq90_inc_transport',
 'STATE',
 'COUNTY_x',
 'FIPS',
 'SVI',
 'STATEFP',
 'COUNTYFP',
 'TRACTCE',
 'AFFGEOID',
 'GEOID',
 'GEOID_2020',
 'COUNTY_y',
 'StateDesc',
 'STATEABBR',
 'LOCATION',
 'E_TOTPOP',
 'M_TOTPOP',
 'E_DAYPOP',
 'SPL_EJI',
 'RPL_EJI',
 'SPL_SER',
 'RPL_SER',
 'SPL_EJI_CBM',
 'RPL_EJI_CBM',
 'E_MINRTY',
 'EPL_MINRTY',
 'SPL_SVM_DOM1',
 'RPL_SVM_DOM1',
 'E_POV200',
 'EPL_POV200',
 'E_NOHSDP',
 'EPL_NOHSDP',
 'E_UNEMP',
 'EPL_UNEMP',
 'E_RENTER',
 'EPL_RENTER',
 'E_HOUBDN',
 'EPL_HOUBDN',
 'E_UNINSUR',
 'EPL_UNINSUR',
 'E_NOINT',
 'EPL_NOINT',
 'SPL_SVM_DOM2',
 'RPL_SVM_DOM2',
 'E_AGE65',
 'EPL_AGE65',
 'E_AGE17',
 'EPL_AGE17',
 'E_DISABL',
 'EPL_DISABL',
 'E_LIM

In [ ]:
import pandas as pd

def extract_columns(input_csv, output_csv, columns_to_extract):
    # Load the CSV file into a DataFrame
    df = pd.read_csv(input_csv)

    # Select the desired columns
    df_selected = df[columns_to_extract]

    # Save to a new CSV file
    df_selected.to_csv(output_csv, index=False)

    print(f"Extracted columns saved to {output_csv}")

# Example usage
extract_columns("merged_fips.csv", "selected_vars.csv",["STATE", "COUNTY_x", "FIPS", "merged_fips.csv", "selected_variables.csv", "pctile_pm25", "pctile_diesel", "pctile_traffic", "SVI", "E_TOTPOP", "EPL_POV200", "EPL_OZONE", "EPL_PM", "EPL_DSLPM", "EPL_TOTCR", "EPL_NPL", "EPL_TRI", "EPL_TSD", "E_PARK", "EPL_ROAD", "EPL_IMPWTR", "EPL_NEHD", "EPL_SMOKE", "EPL_CFLD", "EPL_DRGT","EPL_HRCN", "EPL_RFLD", "EPL_SWND", "EPL_TRND", "Tribe_PCT_Tract", "P_PM25", "P_OZONE", "P_DSLPM", "P_RSEI_AIR", "P_PTRAF", "P_PTSDF", "T_UNEMPPCT", "T_D5_RSEI_AIR", "T_D5_PWDIS"])

selected_vars = pd.read_csv("selected_variables.csv")

KeyError: "['merged_fips.csv', 'selected_variables.csv'] not in index"

In [27]:
import pandas as pd

def extract_columns(input_csv, output_csv, columns_to_extract):
    # Load the CSV file into a DataFrame
    df = pd.read_csv(input_csv)

    # Select only the desired columns (Ensure they exist in the CSV)
    df_selected = df[[col for col in columns_to_extract if col in df.columns]]

    # Save to a new CSV file
    df_selected.to_csv(output_csv, index=False)

    output_csv


input_file = "merged_fips.csv"
output_file = "model_vars.csv"
columns = [
    "STATE", "COUNTY_x", "FIPS", "pctile_pm25", "pctile_diesel", "pctile_traffic", 
    "SVI", "E_TOTPOP", "EPL_POV200", "EPL_OZONE", "EPL_PM", "EPL_DSLPM", "EPL_TOTCR",
    "EPL_NPL", "EPL_TRI", "EPL_TSD", "E_PARK", "EPL_ROAD", "EPL_IMPWTR", "EPL_NEHD", 
    "EPL_SMOKE", "EPL_CFLD", "EPL_DRGT", "EPL_HRCN", "EPL_RFLD", "EPL_SWND", "EPL_TRND", 
    "Tribe_PCT_Tract", "P_PM25", "P_OZONE", "P_DSLPM", "P_RSEI_AIR", "P_PTRAF", "P_PTSDF", 
    "T_UNEMPPCT", "T_D5_RSEI_AIR", "T_D5_PWDIS"
]

# Run the function
extract_columns(input_file, output_file, columns)

# Load the new CSV to verify
model_vars = pd.read_csv(output_file)
model_vars  


,STATE,COUNTY_x,FIPS,pctile_pm25,pctile_diesel,pctile_traffic,SVI,E_TOTPOP,EPL_POV200,EPL_OZONE,...,Tribe_PCT_Tract,P_PM25,P_OZONE,P_DSLPM,P_RSEI_AIR,P_PTRAF,P_PTSDF,T_UNEMPPCT,T_D5_RSEI_AIR,T_D5_PWDIS
0,Georgia,Appling County,13001950100,14,3,6,0.6326,3628,0.8136,0.0,...,0.0,63.0,2.0,5.0,26.0,4.0,0.0,18 %ile,37 %ile,38 %ile
1,Georgia,Appling County,13001950201,16,7,9,0.4667,2392,0.7740,0.0,...,0.0,62.0,2.0,6.0,35.0,8.0,0.0,35 %ile,55 %ile,26 %ile
2,Georgia,Appling County,13001950202,16,7,41,0.8870,2361,0.9459,0.0,...,0.0,63.0,2.0,11.0,54.0,15.0,0.0,70 %ile,83 %ile,29 %ile
3,Georgia,Appling County,13001950301,16,12,40,0.6801,2716,0.8650,0.0,...,0.0,61.0,2.0,13.0,65.0,15.0,0.0,51 %ile,84 %ile,39 %ile
4,Georgia,Appling County,13001950302,16,12,43,0.8420,1759,0.9588,0.0,...,0.0,62.0,2.0,12.0,55.0,15.0,0.0,75 %ile,83 %ile,33 %ile
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2777,Georgia,Worth County,13321950201,35,9,11,0.3951,2241,0.7406,0.0,...,0.0,79.0,20.0,9.0,20.0,11.0,16.0,16 %ile,24 %ile,24 %ile
2778,Georgia,Worth County,13321950202,35,9,52,0.8820,3994,0.8025,0.0,...,0.0,78.0,16.0,9.0,20.0,18.0,0.0,75 %ile,29 %ile,0 %ile
2779,Georgia,Worth County,13321950400,25,4,21,0.7218,4800,0.6972,0.0,...,0.0,77.0,17.0,7.0,15.0,10.0,0.0,49 %ile,20 %ile,0 %ile
2780,Georgia,Worth County,13321950500,33,5,38,0.8841,4293,0.7202,0.0,...,0.0,79.0,22.0,9.0,7.0,16.0,15.0,76 %ile,12 %ile,24 %ile
